# Borrower Reliability Research


The customer is the bank's credit department. It is necessary to determine whether the marital status and number of children of a client affect the timely repayment of a loan. The input data from the bank is statistics on the clients' creditworthiness.
The results of the study will be taken into account when building a credit scoring model — a special system that assesses the ability of a potential borrower to repay the loan to the bank.


## General information about data

Importing the pandas library and studying the dataset.


In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


There are missing values in the columns, but otherwise, the dataset seems fine: the data types are appropriate, and the column names comply with the syntax.


## Data preprocessing

### Removing missing values

In [4]:
# the number of missing values for each column
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

I will handle the missing values in the total_income column, which contains income data. The type of employment has the greatest impact on the amount of income, so the missing values in this column should be filled with the median value for each type from the income_type column. For example, a person with the employment type employee should have the missing value in the total_income column filled with the median income among all records with the same employment type.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Handling of anomalous values

The data may contain anomalies — values that do not reflect reality and appeared due to some error. One such artifact is the negative number of days of employment in the days_employed column. I will replace all negative values with positive ones using the abs() method.


In [6]:
data['days_employed'] = data['days_employed'].abs()

I will output the median value of employment duration days_employed in days for each type of employment.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

Two types (unemployed - безработный and retirees - пенсионер) have abnormally large values. Correcting such values is difficult, so I will leave them as they are. Moreover, this column will not be needed for the study.

I will output the list of unique values in the children column.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In the 'children' column, there are two anomalous values. I will remove the rows containing these anomalous values from the 'data' dataframe.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

I will fill the missing values in the days_employed column with the median values for each employment type income_type.

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Changing data types


I will replace the float data type in the total_income column.

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Handling duplicates

I will handle implicit duplicates in the education column. This column contains the same values recorded differently: using uppercase and lowercase letters. I will convert them to lowercase. I will check the other columns as well.

In [14]:
data['education'] = data['education'].str.lower()

I will display the number of duplicate rows in the data. If such rows are present, I will delete them.


In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Data categorization

Based on the ranges specified below, I will create a column total_income_category in the dataframe data with the following categories:

- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 and above — 'A'.

For example, a borrower with an income of 25000 should be assigned the category 'E', and a client receiving 235000 should be assigned the category 'B'.


In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
# unique loan purposes from the purpose column
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

I will create a function that, based on the data from the purpose column, will form a new column purpose_category with the following categories:
'операции с автомобилем',
'операции с недвижимостью',
'проведение свадьбы',
'получение образования'.

For example, if the purpose column contains the substring 'на покупку автомобиля', then the purpose_category column should contain the string 'операции с автомобилем'.

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Data analysis and answering business questions

#### Is there a correlation between the number of children and timely loan repayment?

In [22]:

kids_pivot = data.pivot_table(index='children', values = 'debt', aggfunc= ['count', 'sum', 'mean']) # counting the number of borrowers, the number of debtors, and the proportion of debtors by the 'debt' column, grouped by the number of children

kids_pivot.columns = ['Всего кредитополучателей', 'Доля должников', 'Всего должников'] # renaming the columns to more understandable names

kids_pivot.sort_values(by='Всего кредитополучателей', ascending=False) # sorted the table in descending order by the "Total Borrowers" column



,Всего кредитополучателей,Доля должников,Всего должников
children,,,
0,14091,1063,0.075438
1,4808,444,0.092346
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


**Conclusion:** 

Clients without children take out the most loans, but they also repay them the best. The more children a person has, the less likely they are to take out a loan. However, if a client with children does take out a loan, there is a high probability that they will not repay it.

#### Is there a correlation between marital status and timely loan repayment?

In [23]:
family_pivot = data.pivot_table(index='family_status', values = 'debt', aggfunc= ['count', 'sum', 'mean']) # counting the number of borrowers, the number of debtors, and the proportion of debtors by the 'debt' column, grouped by marital status
family_pivot.columns = ['Всего кредитополучателей', 'Доля должников', 'Всего должников']
family_pivot.sort_values(by='Всего кредитополучателей', ascending=False)

,Всего кредитополучателей,Доля должников,Всего должников
family_status,,,
женат / замужем,12261,927,0.075606
гражданский брак,4134,385,0.093130
Не женат / не замужем,2796,273,0.097639
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246


**Conclusion:** 

Clients who were/are married repay their loans better than those who were not legally bound. However, this conclusion does not clarify how many divorced clients are among those in a civil union.

Regarding unmarried clients: there are likely many young people with unstable incomes among them, which is why the repayment rate is so low. More research is needed here.

#### Is there a correlation between income level and timely loan repayment?

In [24]:
total_income_pivot = data.pivot_table(index='total_income_category', values = 'debt', aggfunc= ['count', 'sum', 'mean']) # counting the number of borrowers, the number of debtors, and the proportion of debtors by the 'debt' column, grouped by income
total_income_pivot.columns = ['Всего кредитополучателей', 'Доля должников', 'Всего должников']

total_income_pivot.sort_values(by='Всего кредитополучателей', ascending=False)

,Всего кредитополучателей,Доля должников,Всего должников
total_income_category,,,
C,15921,1353,0.084982
B,5014,354,0.070602
D,349,21,0.060172
A,25,2,0.080000
E,22,2,0.090909


**Conclusion**

Clients with average earnings take out the most loans. They do not repay them very poorly, but they could do better. Clients with earnings from 200,001 to 1,000,000 take out loans much less frequently but repay them better. The higher the income, the lower the proportion of debtors.

#### How do different loan purposes affect timely repayment?

In [25]:
purpose_pivot = data.pivot_table(index='purpose_category', values = 'debt', aggfunc= ['count', 'sum', 'mean']) #counting the number of borrowers, the number of debtors, and the proportion of debtors by the 'debt' column, grouped by loan purposes
purpose_pivot.columns = ['Всего кредитополучателей', 'Доля должников', 'Всего должников']
purpose_pivot.sort_values(by='Всего кредитополучателей', ascending=False) 

,Всего кредитополучателей,Доля должников,Всего должников
purpose_category,,,
операции с недвижимостью,10751,780,0.072551
операции с автомобилем,4279,400,0.093480
получение образования,3988,369,0.092528
проведение свадьбы,2313,183,0.079118


**Conclusion:** 

People who invest in real estate are the most reliable borrowers. This makes sense, as real estate is usually purchased when life is more or less stable. There are few repayments for education loans because young people take out these loans, and they don't have much money yet. I've never owned a car, so I don't have specific ideas on why auto loans have low repayment rates. Perhaps car repairs are more expensive than not repaying the loan on time.

#### What are possible reasons for missing values in the original data

It is important to note human and technical factors here. Clients might not have provided the data. This information might not be required in some cases. Problems could have arisen during the data loading and extraction stages. An incorrect query or table merging could also be a reason. Quantitative variables contain numerical values within a certain range. Missing values in such variables can be filled with the mean or median. In the dataset, there are missing values in the columns with income and work experience. In this case, some values may stand out significantly among the majority, and the mean value may not accurately characterize the data. It is better to use the median value.

### Genaral conclusion

#### Data Preprocessing

1. Studied the data structure.
2. Removed missing values in the columns days_employed and total_income. In days_employed, removed anomalies (eliminated negative values) and filled in the missing values with the median value of the column. In total_income, filled in the missing values with the median value of the column.
3. Removed anomalies in the children column: deleted values -1 and 20.
4. Changed the data type in the total_income column from float to int.
5. Removed duplicates from the education column.
6. Added a column total_income_category with income categories and a column purpose_category with purpose categories.

#### Calculations

1. Counted the number of borrowers, the number of debtors, and the proportion of debtors by the debt column, grouped by marital status, number of children, purposes, and income.
2. Generated a summary table for each grouping with the columns 'Total Borrowers', 'Total Debtors', and 'Proportion of Debtors'. Sorted the table in descending order by the 'Total Borrowers' column.

#### Conclusions

1. Clients without children take out the most loans but repay them well. The more children, the higher the likelihood that the person will not repay the loan.
2. Unmarried clients, likely due to their youth, repay loans less frequently. If a person has been married, the likelihood of repaying the loan is higher, probably due to older age.
3. Clients with average earnings take out the most loans. Although their repayment is not perfect, I believe they are still the bank's target audience.
4. People who invest in real estate are the most reliable borrowers. There are few repayments for education loans because young people take out these loans. Auto loans also have poor repayment rates: maintaining a car is expensive, making it difficult to repay the bank.
5. To make more accurate predictions about clients who most frequently default on loans, further data research is needed. It is necessary to examine how young people repay loans, whether there is a correlation between couples, the number of children, and the degree of debt repayment.

By income, the ideal client is category B (200,001-1,000,000), the most risky is category C (50,001-200,000). As recommendations, one could suggest a different categorization by income. In the questionnaire, make the fields for income and work experience mandatory to fill out, and add auto-validation for the format of the data entered by the user.
